In [1]:
using JLD2
using Statistics
using Revise
using Plots
using Laplacians
using Printf


In [2]:
@load "sachdeva_star_all.jl.split02merge02.jld2"
dic1 = copy(dic)

Dict{Any, Any} with 84 entries:
  "lamg_build"        => [0.24798, 0.439223, 0.957284, 1.4367, 2.21285, 3.53374…
  "acM2C2_err"        => [6.76857e-9, 9.6469e-9, 6.93641e-9, 7.47933e-9, 6.5201…
  "hypre_solve"       => [0.22, 0.76, 2.77, 6.66, 11.46, 18.32, 26.95, 38.82, 5…
  "acM2C2_solve"      => [0.0453529, 0.215648, 0.664904, 1.43216, 2.38298, 4.25…
  "icc2_tot"          => [0.271866, 0.383913, 0.707252, 1.24781, 2.06733, 3.282…
  "acM2C2_tot"        => [0.259798, 1.0003, 2.5187, 5.65618, 9.69102, 15.9491, …
  "ac_lst_build"      => [0.0976722, 0.478637, 1.09675, 2.8267, 4.88578, 7.4235…
  "petsc_hypre_build" => [0.526289, 2.42528, 7.55318, 18.2163, 37.366, 68.7379,…
  "lamg_its"          => [4.0, 4.0, 6.0, 4.0, 13.0, 10.0, 6.0, 11.0, 8.0, 8.0, …
  "lamg_solve"        => [0.085986, 0.137766, 0.332293, 0.368882, 1.79307, 2.09…
  "hypre_its"         => [100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.…
  "cmg_solve"         => [0.215778, 0.305125, 0.497897, 0.836569, 1.29838, 1.

In [3]:
function sd(d,x)
    #scale = exp10(ceil(log10(abs(x)))-d)
    #return round(x/scale)*scale
    return round(x, sigdigits=d)
end        

#constant d = 3
ff(s) = (@sprintf "%.3g" sd(3,s))
nf(s) = (@sprintf "%d" sd(3,s))

ffK(s) = (@sprintf "%.3g" sd(3,s)/1000)
nfK(s) =  round(Int64, sd(3,s)/1000)

ffe8(s) = (@sprintf "%.3g" sd(3,s)*1e8)
ffM(s) = (@sprintf "%.3g" sd(3,s)/1e6)
nfM(s) =  round(Int64, sd(3,s)/1e6)
ffe6(s) = (@sprintf "%.3g" sd(3,s)*1e6)
ffe5(s) = (@sprintf "%.3g" sd(3,s)*1e5)


ffe5 (generic function with 1 method)

In [4]:
function labelTvn(tvn, err)
    if err <= 1e-8
        return ffe6(tvn)
    elseif err < 1e-4
        return string(ffe6(tvn), "\\tnote{*}")
    elseif err < 1
        return string(ffe6(tvn), "\\tnote{**}")
    else
        return "Inf"
    end
end

function printSolverCell(d, solver, i)
    if haskey(d, string(solver, "_tot"))
        itr = d[string(solver, "_its")][i]
        ne = d["ne"][i]
        solveVn = d[string(solver, "_solve")][i] / ne
        totalVn = d[string(solver, "_tot")][i] / ne
        err = d[string(solver, "_err")][i]
        return string("&", nf(itr), "&", labelTvn(solveVn, err), "&", labelTvn(totalVn, err), "&",ffe8(err))
    else
        return "&N/A&N/A&N/A&N/A"
    end
end

function printOneLine(d, solvers, i)
    allCells = [printSolverCell(d, solver, i) for solver in solvers]
    return string(100 + 50*(i-1),"&",nfK(d["ne"][i]), "&", nfK(d["nv"][i]), join(allCells), "\\\\")
end

function printAllLines(d, solvers)
    for i in 1:length(d["testName"])
        println(printOneLine(d, solvers, i))
    end
end

printAllLines (generic function with 1 method)

In [5]:
string(ffe6(dic1["ac_err"][10]), "\\tnote{**}")

"188\\tnote{**}"

In [6]:
solvers1 = ["ac", "ac-s2m2", "cmg2"]
printAllLines(dic1, solvers1)

100&500&5&83&0.226&0.398&0.9&28&0.0793&0.478&0.64&7&0.4&1.18&0.000943\\
150&1690&11&149&0.502&0.748&0.96&34&0.117&0.581&0.632&7&0.18&0.62&0.000857\\
200&4000&20&167&0.682&0.931&0.955&37&0.155&0.716&0.877&7&0.149&0.487&0.00102\\
250&7810&31&241&1.07&1.43&0.998&38&0.169&0.703&0.98&7&0.136&0.475&0.00131\\
300&13500&45&289&1.33&1.59&0.88&39&0.179&0.719&0.854&7&0.125&0.436&0.00173\\
350&21400&61&355&1.65&1.97&0.99&40&0.188&0.733&0.996&7&0.122&0.424&0.00177\\
400&32000&80&459&2.12&2.44&0.967&40&0.186&0.718&0.985&7&0.12&0.41&0.00197\\
450&45600&101&486&2.27&2.58&0.999&43&0.202&0.745&0.902&7&0.119&0.401&0.00177\\
500&62500&125&452&2.08&2.4&0.944&43&0.198&0.747&0.808&7&0.118&0.399&0.00193\\
550&83200&151&149&0.698\tnote{**}&1.02\tnote{**}&1.88e+04&41&0.192&0.753&0.792&7&0.118&0.4&0.00315\\
600&108000&180&773&3.55&3.93&0.997&45&0.208&0.849&0.741&7&0.117&0.395&0.00251\\
650&137000&211&953&4.43\tnote{*}&4.76\tnote{*}&2.4&44&0.201&1.04&0.687&7&0.118&0.404&0.00319\\
700&172000&245&478&2.19\tnote{*}&

In [7]:
solvers2 = ["hypre", "petsc_hypre", "icc2"]
printAllLines(dic1, solvers2)

100&500&5&100&0.44\tnote{*}&0.84\tnote{*}&225&2&0.032\tnote{**}&1.08\tnote{**}&2.76e+04&1&0.428&0.544&4.44e-06\\
150&1690&11&100&0.45\tnote{*}&0.995\tnote{*}&493&2&0.0341\tnote{**}&1.47\tnote{**}&2.63e+04&1&0.125&0.227&8.25e-08\\
200&4000&20&100&0.692\tnote{*}&1.4\tnote{*}&2.87e+03&2&0.0376\tnote{**}&1.93\tnote{**}&2.09e+04&1&0.0649&0.177&5.42e-07\\
250&7810&31&100&0.852\tnote{*}&1.73\tnote{*}&15.2&1&0.041\tnote{**}&2.37\tnote{**}&1.68e+04&1&0.04&0.16&8.73e-07\\
300&13500&45&100&0.849\tnote{**}&1.89\tnote{**}&3.91e+04&2&0.0427\tnote{**}&2.81\tnote{**}&1.56e+04&1&0.0315&0.153&1.37e-05\\
350&21400&61&100&0.855\tnote{**}&2.07\tnote{**}&1.14e+06&5&0.106\tnote{*}&3.31\tnote{*}&4.21e+03&1&0.0265&0.153&1.38e-06\\
400&32000&80&100&0.842\tnote{*}&2.22\tnote{*}&1.54e+03&1&0.0394\tnote{**}&3.69\tnote{**}&1.2e+04&1&0.0243&0.163&1.54e-05\\
450&45600&101&100&0.852\tnote{**}&2.4\tnote{**}&2.71e+06&2&0.0388\tnote{**}&4.13\tnote{**}&1.21e+04&1&0.0224&0.168&3.07e-06\\
500&62500&125&100&0.848\tnote{**}&2